# CI/CD for TFX pipelines

## Learning Objectives

1.  Develop a CI/CD workflow with Cloud Build to build and deploy TFX pipeline code.
2.  Integrate with Github to automatically trigger pipeline deployment with source code repository changes.

In this lab, you will walk through authoring a Cloud Build CI/CD workflow that automatically builds and deploys a TFX pipeline. You will also integrate your workflow with GitHub by setting up a trigger that starts the workflow when a new tag is applied to the GitHub repo hosting the pipeline's code.



## Understanding the Cloud Build workflow
Review the `cloudbuild_vertex.yaml` file to understand how the CI/CD workflow is implemented and how environment specific settings are abstracted using **Cloud Build** variables.

The **Cloud Build** CI/CD workflow automates the steps you walked through manually during the second lab of this series:
1. Builds the custom TFX image to be used as a runtime execution environment for TFX components
1. Pushes the custom TFX image to your project's **Container Registry**
1. Compiles and run the TFX pipeline on Vertex pipelines

The **Cloud Build** workflow configuration uses both standard and custom [Cloud Build builders](https://cloud.google.com/cloud-build/docs/cloud-builders). The custom builder encapsulates **TFX CLI**. 


## Configuring environment settings

Navigate to [Vertex Pipelines](https://console.cloud.google.com/vertex-ai/pipelines) page in the Google Cloud Console.


You may need to open CloudShell and run the following command to allow CloudBuild to deploy a pipeline on Vertex:

```bash
export PROJECT_ID=$(gcloud config get-value core/project)

PROJECT_NUMBER=$(gcloud projects describe $PROJECT_ID --format="value(projectNumber)")

CLOUD_BUILD_SERVICE_ACCOUNT="${PROJECT_NUMBER}@cloudbuild.gserviceaccount.com"

gcloud projects add-iam-policy-binding $PROJECT_ID \
  --member serviceAccount:$CLOUD_BUILD_SERVICE_ACCOUNT \
  --role roles/editor
```

### Configure environment settings

In [ ]:
REGION = "us-central1"
PROJECT_ID = !(gcloud config get-value core/project)
PROJECT_ID = PROJECT_ID[0]
ARTIFACT_STORE = f"gs://{PROJECT_ID}"

## Creating the TFX CLI builder

### Review the Dockerfile for the TFX CLI builder

In [ ]:
!cat tfx-cli_vertex/Dockerfile

### Build the image and push it to your project's Container Registry

**Hint**: Review the [Cloud Build](https://cloud.google.com/cloud-build/docs/running-builds/start-build-manually#gcloud) gcloud command line reference for builds submit. Your image should follow the format `gcr.io/[PROJECT_ID]/[IMAGE_NAME]`. Note the source code for the tfx-cli is in the directory `./tfx-cli_vertex`. It has an helper function `tfx_pipeline_run.py` to run a compiled pipeline on Vertex.

In [ ]:
IMAGE_NAME = "tfx-cli_vertex"
IMAGE_URI = f"gcr.io/{PROJECT_ID}/{IMAGE_NAME}"
IMAGE_URI

In [ ]:
# TODO: Your gcloud command here to build tfx-cli and submit to Container Registry.
!gcloud builds submit --timeout=15m --tag {IMAGE_URI} {IMAGE_NAME}

## Exercise: manually trigger CI/CD pipeline run with Cloud Build

You can manually trigger **Cloud Build** runs using the `gcloud builds submit` command.

See the [documentation](https://cloud.google.com/sdk/gcloud/reference/builds/submit) for pass the `cloudbuild_vertex.yaml` file and the substitutions.

In [ ]:
SUBSTITUTIONS = f"_REGION={REGION}"

In [ ]:
# TODO: write gcloud builds submit command to trigger manual pipeline run.
!gcloud builds submit . --timeout=2h --config cloudbuild_vertex.yaml --substitutions {SUBSTITUTIONS}

## Exercise: Setting up GitHub integration

In this exercise you integrate your CI/CD workflow with **GitHub**, using [Cloud Build GitHub App](https://github.com/marketplace/google-cloud-build). 
You will set up a trigger that starts the CI/CD workflow when a new tag is applied to the **GitHub** repo managing the  pipeline source code. You will use a fork of this repo as your source GitHub repository.

### Create a fork of this repo

#### [Follow the GitHub documentation](https://help.github.com/en/github/getting-started-with-github/fork-a-repo) to fork this repo

#### Create a Cloud Build trigger

Connect the fork you created in the previous step to your Google Cloud project and create a trigger following the steps in the [Creating GitHub app trigger](https://cloud.google.com/cloud-build/docs/create-github-app-triggers) article. Use the following values on the **Edit trigger** form:

|Field|Value|
|-----|-----|
|Name|[YOUR TRIGGER NAME]|
|Description|[YOUR TRIGGER DESCRIPTION]|
|Event| Tag|
|Source| [YOUR FORK]|
|Tag (regex)|.\*|
|Build Configuration|Cloud Build configuration file (yaml or json)|
|Cloud Build configuration file location|/ notebooks/tfx_pipelines/cicd/solutions/cloudbuild_vertex.yaml|

Use the following values for the substitution variables:

|Variable|Value|
|--------|-----|
|_REGION|[YOUR GCP_REGION]|


#### Trigger the build

To start an automated build [create a new release of the repo in GitHub](https://help.github.com/en/github/administering-a-repository/creating-releases). Alternatively, you can start the build by applying a tag using `git`. 
```
git tag [TAG NAME]
git push origin --tags
```


#### Verify triggered build in Cloud Build dashboard

After you see the pipeline finish building on the Cloud Build dashboard, return to [Vertex Pipelines](https://console.cloud.google.com/vertex-ai/pipelines) in the console to check that pipeline completed successfully.

# License

Copyright 2021 Google Inc. All Rights Reserved.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

           http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.